# ICEPY

## Introduction

Welcome to icepy ! 

Icepy is a python library for IDA ICE 4.8. It makes use of IDA ICE API functions to build idm models, change parameter, simulate and output results.It takes full advantage of the rich data structure and idioms that are avaliable in python. The power of using a scripting language allows you to do the following:

* Make a large number of changes in an idm model with a few lines of icepy code.
* Make changes to windows, doors, window-to-wall ratios in an idm model.
* Apply an IDA ICE simulation run (Special license required from Equa).
* Apply multiple simulation runs.
* Read data from the output reports of an IDA ICE simulation.

# Start here
This tutorial is intended as a walk-through for complete beginners to icepy.

In outline, you will:

* Install python3 32-bit and icepy
* Install IDA ICE 4.8 (if you haven’t already)
* Enable IDAMessageBrokerService 
* Create a simple test building based on baseline building
* Run a simulation in IDA ICE 4.8
* Read output results

## Install python3 and icepy

IDA ICE 4.8 is developed in Windows as a 32 bit program. You can download [python3](https://www.python.org/downloads/) in your PC. Please make sure a 32-bit version is installed. 

After successfully installed python3 32-bit, you can install icepy with the command line.  

## Install IDA ICE

IDA ICE is a software developed for building energy simulation by Equa. If you haven't have it installed, please contact [Equa](https://www.equa.se/en/). It is worth mentioning, the simulation function via IDA ICE API requires a spcieal version of license. Please make sure you get the correct license.

## Enable IDAMessageBrokerService

Fllow steps:
* Windows Services Management Tool (system level)  
* start 'IDAMessageBrokerService 
* Optional: Startup Type: Automatic

## Create a simple testing model

### Step 1 : Replate the path to IDA ICE installation folder

You’ll need to replace the path to the IDAICE installation folder with the appropriate path.

For example, on Windows, this is usually C:\\IDAICE\\bin\\

In [1]:
from icepy.config import ICE

In [2]:
ICE.setice("D:\\idaice\\bin\\")
ICE.ice_path

'D:\\idaice\\bin\\'

### Step 2 : Import an idm model

For instance, we import an example idm model. You can download from [icepy github page](https://github.com/chenglongfu11/icepy/tree/master/buildings). 

In [3]:
folder = 'D:\\ide_mine\\changing\\'
base_model = 'ut1_1'

The wall width list and the building height are provided along with the idm model. 

In [4]:
wall_width_list = [4, 7.5, 11.63, 10.27, 18.07, 20.587, 6.543, 8.72, 6.5176, 4, 6.48, 2.59, 31, 4, 9.886, 8.554,
                       13.899, 11.15, 9.98]
building_ht = 22.5 

You can set desired the floor height, the number of floors. 

In [5]:
floor_ht = 3                 
num_floors = 5

Let's import the idm model

In [6]:
from icepy.modeleditor import IDM
idm = IDM(folder, base_model)

In [7]:
# Building object
building = idm.openfile(base_model)

### Step 2: Clone zones

In the idm model, one zone with 2.6 height for the ground floor is included. We assume there is one zone for each floor. So we clone zones. Note that the collection of zones should not overflow this building.

In [8]:
result, new_file = idm.zoneclone(floor_ht, num_floors)
result

Changing ceiling height
The current CEILING-HEIGHT value is 2.600000, and the type is <class 'float'>
True
True
True
True


True

A new file name is generated, then we can save the new idm file with cloned zones.

In [9]:

idm.saveas(new_file)

Successfully save file : D:\ide_mine\changing\ut1_1_5floor.idm


True

### Step 3: Add windows and doors



In [10]:
# Windows
wins = []
win_dx = 1.5
win_dy = 1.2
for i in range(len(wall_width_list)):
    width = wall_width_list[i]
    if width > win_dx + 2 and floor_ht > win_dy:  # to make sure it is in the zone
        win = {}
        win['wall_name'] = 'WALL_' + str(i + 1)
        win['win_x'] = str(width / 2 - win_dx / 2)
        win['win_y'] = '0.8'
        win['win_dx'] = '1.5'
        win['win_dy'] = '1.2'
        wins.append(win)

In [11]:
# Detailed window
win2 = {}
win2['wall_name'] = 'WALL_13'
win2['win_x'] = '26.015'
win2['win_y'] = '0.8'
win2['win_dx'] = '3'
win2['win_dy'] = '1.8'
win2['detailed'] = 1   # true 1   false 0
win2['glazing'] = 1
wins.append(win2)

In [12]:
# Doors
doors = []
door = {}
door['wall_name'] = 'WALL_6'
door['door_x'] = '3.21'
doors.append(door)

Now, we have designed several windows and doors to add to the building model. You can also read csv files like examples in [icepy github page](https://github.com/chenglongfu11/icepy/tree/master/buildings). 

We then apply these components into the model. Ideally, they are uniformly distributed on eahc floor.

In [13]:
idm.applyscript(num_floors, wins, doors)

((CE-ZONE :N "Zone 1")( (AGGREGATE :N THERMAL-BRIDGES) (:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N WIN-PERIM-LEN)(:PAR :N DOOR-PERIM-LEN))( (ENCLOSING-ELEMENT :N WALL_1)(:ADD (CE-WINDOW :N "Window" :T WINDOW)  (:PAR :N X :V 1.25) (:PAR :N Y :V 0.8) (:PAR :N DX :V 1.5) (:PAR :N DY :V 1.2) (:PAR :N LIGHT-LEVEL) (:PAR :N CD_LO)  )) ( (ENCLOSING-ELEMENT :N WALL_2)(:ADD (CE-WINDOW :N "Window" :T WINDOW)  (:PAR :N X :V 3.0) (:PAR :N Y :V 0.8) (:PAR :N DX :V 1.5) (:PAR :N DY :V 1.2) (:PAR :N LIGHT-LEVEL) (:PAR :N CD_LO)  )) ( (ENCLOSING-ELEMENT :N WALL_3)(:ADD (CE-WINDOW :N "Window" :T WINDOW)  (:PAR :N X :V 5.065) (:PAR

Apply_script message:  <ctypes.c_char_Array_5000 object at 0x1118BE38>


You can check the new model in the IDA ICE now. 

### Step 4: Run simulation

It takes a while to run the simulation. You can check the process in IDA ICE. 

In [15]:
# Run Simulation
idm.simulation_bld(new_file)

---------------
Check point before simulation IDLE
Simulation begins. 
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  20.809334754943848
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  41.015286445617676
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  61.224011182785034
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  81.4644546508789
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  101.7041494846344
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  122.14698314666748
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  142.75225067138672
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  163.43998527526855
Simulation tracking ut1_1_5floor.idm : SIMULATING . Time consumed:  183.65881729125977
======> Simulation complete <======
Total time 213.72576355934143
Simulation time 191.1966404914856
---------------


### Step 5: Generate html report
The function html_report_bld reqires a target folder and a name of the html report.

In [16]:
idm.html_report_bld(folder,'111')

Generated the html report in  D:\ide_mine\changing\111.html


'D:\\ide_mine\\changing\\111.html'

### Step 6: Save and close

In [17]:
idm.save_close()